In [4]:
import cbpro # DO NOT UPDATE REQUESTS; CBPRO ONLY WORKS WITH 2.13
import pandas as pd
import datetime
import csv
import time
cb_public = cbpro.PublicClient()

In [5]:
# ISO 8601 - YYYY-MM-DD HH:MM:SS
# create list of start dates
start = datetime.datetime(2019, 12, 19, 00, 00, 00) # first wuhan patients as per cdc
end = datetime.datetime(2021, 12, 20)
start_date_list = pd.date_range(start, end, freq="D")

# create list of end dates
start = datetime.datetime(2019, 12, 19, 23, 59, 00)
end = datetime.datetime(2021, 12, 20)
end_date_list = pd.date_range(start, end, freq="D")

# CHOOSE PAIR TO SCRAPE
crypto_pair = ["ETH-USD", "BTC-USD"]

In [6]:
# get_product_historic_rates is a public endpoint
# "We throttle public endpoints by IP: 10 requests per second, up to 15 requests per second in bursts. Some endpoints may have custom rate limits.""
# granularity = tick frequency in seconds (60 = 1 min, 300 = 5 min, 900 = 15 min, 3600 = 1 hr, 21600 = 6 hrs, 86400 = 24 hrs)
# If either one of the start or end fields are not provided then both fields will be ignored. If a custom time range is not declared then one ending now is selected.
# The maximum number of data points for a single request is 300 candles

# time: bucket start time
# low: lowest price during the bucket interval
# high: highest price during the bucket interval
# open: opening price (first trade) in the bucket interval
# close: closing price (last trade) in the bucket interval
# volume: volume of trading activity during the bucket interval

# Runtime: 19 min 1.1s
header = ["time", "low", "high", "open", "close", "volume"]
#time_period = {"hr_24" : 86400,"hr_1" : 3600, "min_15" : 900, "min_5" : 300}
time_period = {"hr_24" : 86400,"hr_1" : 3600}
for pair in crypto_pair:
    for period in time_period:
        # create csv files with header
        with open(f'{pair}_{period}.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)
        for start_date, end_date in zip(start_date_list, end_date_list):
            historical = pd.DataFrame(cb_public.get_product_historic_rates(product_id=pair, start = start_date, end = end_date, granularity = time_period[period]))
            historical.columns = header
            historical.sort_values(by='time', ascending=True, inplace=True)
            # don't write multiple headers
            historical.to_csv(f'{pair}_{period}.csv', header= False, index=False, mode='a')
            time.sleep(.1) # min value is .1 b/c public endpoint rate limit = 10/s

300 data points * 5 minute ticks = 25 hour time frame (1500 minutes)

10 requests per second = 10.416 days (250 hours)

730 days / 10 = 73 seconds total